In [1]:
import os
import sys
from mmengine import MMLogger
from mmengine.config import Config
from mmengine.dist import init_dist
from mmengine.registry import init_default_scope
from mmengine.utils import mkdir_or_exist
from mmdet.utils.benchmark import (DataLoaderBenchmark, DatasetBenchmark, InferenceBenchmark)


import mmdet

In [2]:
import os
import sys

custom_path = '/Ziob/343312/AVD-Anonymizer/CrossKD'
if custom_path not in sys.path:
    sys.path.insert(0, custom_path)

print("Python Sys Path:")
for path in sys.path:
    print(path)

from mmengine.config import Config
from mmdet.utils.benchmark import InferenceBenchmark
from mmengine.registry import init_default_scope

init_default_scope('mmdet')

# Try importing using a path relative to the added sys.path
from mmdet.models.detectors import crosskd_fcos
CrossKDFCOS = crosskd_fcos.CrossKDFCOS

Python Sys Path:
/Ziob/343312/miniconda/envs/crosskd/lib/python38.zip
/Ziob/343312/miniconda/envs/crosskd/lib/python3.8
/Ziob/343312/miniconda/envs/crosskd/lib/python3.8/lib-dynload

/Ziob/343312/miniconda/envs/crosskd/lib/python3.8/site-packages
/Ziob/343312/AVD-Anonymizer/CrossKD
/tmp/tmppba8n3ob


In [8]:
config_file = '/Ziob/343312/AVD-Anonymizer/CrossKD/configs/fcos/fcos_r50-caffe_fpn_gn-head_2x_coco.py'
checkpoint_path = '/Ziob/343312/AVD-Anonymizer/CrossKD/work_dirs/fcos_r50-caffe_fpn_gn-head_2x_coco/epoch_24.pth'

# config_file = '/Ziob/343312/AVD-Anonymizer/CrossKD/configs/crosskd/crosskd_r50_fcos_r101-2x-ms_caffe_fpn_gn-head_2x_ms_coco.py'
# checkpoint_path = '/Ziob/343312/AVD-Anonymizer/CrossKD/work_dirs/crosskd_r50_fcos_r101-2x-ms_caffe_fpn_gn-head_2x_ms_coco/epoch_24.pth'

cfg = Config.fromfile(config_file)

In [9]:

absolute_data_path = '/Ziob/343312/AVD-Anonymizer/CrossKD/data/pp4av_dataset'

if 'test_dataloader' in cfg:
    if hasattr(cfg.test_dataloader.dataset, 'data_root'):
        cfg.test_dataloader.dataset.data_root = absolute_data_path
    # For COCO-style datasets, we also need to update data_prefix if it exists
    if hasattr(cfg.test_dataloader.dataset, 'data_prefix'):
        if isinstance(cfg.test_dataloader.dataset.data_prefix, dict):
            for key in cfg.test_dataloader.dataset.data_prefix:
                if key == 'img':
                    cfg.test_dataloader.dataset.data_prefix[key] = os.path.join(absolute_data_path, 'val')
        else:
            cfg.test_dataloader.dataset.data_prefix = os.path.join(absolute_data_path, 'val')
    if hasattr(cfg.test_dataloader.dataset, 'ann_file'):
        cfg.test_dataloader.dataset.ann_file = os.path.join(absolute_data_path, 'annotations/instances_val.json')


init_default_scope(cfg.get('default_scope', 'mmdet'))
logger = MMLogger.get_instance('mmdet', log_level='INFO')

In [10]:
task = "inference" 
repeat_num = 1
max_iter = 100  
log_interval = 20
num_warmup = 5
dataset_type = "test"

#  Enable conv-bn fusion for potentially faster inference
fuse_conv_bn = True
distributed = False

print(f"Task: {task}")
print(f"Checkpoint path: {checkpoint_path}")
print(f"Dataset type: {dataset_type}")
print(f"Max iterations: {max_iter}")
print(f"Log interval: {log_interval}")
print(f"Fuse Conv-BN: {fuse_conv_bn}")

benchmark = InferenceBenchmark(
    cfg,
    checkpoint_path,
    distributed,
    fuse_conv_bn,
    max_iter,
    log_interval,
    num_warmup,
    logger=logger
)

results = benchmark.run(repeat_num)

print("\nDetailed Benchmark Results:")
if hasattr(results, 'items'):
    for key, value in results.items():
        print(f"{key}: {value}")
else:
    print(results)

Task: inference
Checkpoint path: /Ziob/343312/AVD-Anonymizer/CrossKD/work_dirs/fcos_r50-caffe_fpn_gn-head_2x_coco/epoch_24.pth
Dataset type: test
Max iterations: 100
Log interval: 20
Fuse Conv-BN: True
05/26 23:42:56 - mmengine - INFO - before build: 
05/26 23:42:56 - mmengine - INFO - (GB) mem_used: 12.28 | uss: 0.33 | pss: 0.34 | total_proc: 1


TypeError: class `FCOS` in mmdet/models/detectors/fcos.py: __init__() got an unexpected keyword argument 'pretrained'

In [40]:
task = "inference" 
repeat_num = 1
max_iter = 100  
log_interval = 20
num_warmup = 5
dataset_type = "test"
#  Enable conv-bn fusion for potentially faster inference
fuse_conv_bn = True
distributed = False

print(f"Task: {task}")
print(f"Checkpoint path: {checkpoint_path}")
print(f"Dataset type: {dataset_type}")
print(f"Max iterations: {max_iter}")
print(f"Log interval: {log_interval}")
print(f"Fuse Conv-BN: {fuse_conv_bn}")

# Fix for the pretrained parameter issue
# Let's create a custom InferenceBenchmark class that fixes the issue

class CustomInferenceBenchmark(InferenceBenchmark):
    def _init_model(self, checkpoint: str, is_fuse_conv_bn: bool) -> nn.Module:
        """Initialize the model with the pretrained parameter removed."""
        # Make a copy of the model config to avoid side effects
        model_cfg = copy.deepcopy(self.cfg.model)
        
        # Remove the pretrained parameter if it exists
        if hasattr(model_cfg, 'pretrained'):
            delattr(model_cfg, 'pretrained')
        elif isinstance(model_cfg, dict) and 'pretrained' in model_cfg:
            del model_cfg['pretrained']
            
        # Build the model without the pretrained parameter
        model = MODELS.build(model_cfg)
        
        # Load checkpoint if provided
        if checkpoint:
            load_checkpoint(model, checkpoint, map_location='cpu')
        
        # Apply fuse_conv_bn if specified
        if is_fuse_conv_bn:
            model = fuse_conv_bn_eval(model)
            
        model.eval()
        return model

# Use our custom benchmark class
benchmark = CustomInferenceBenchmark(
    cfg,  # Use the original config - our custom class will handle the pretrained parameter
    checkpoint_path,
    distributed,
    fuse_conv_bn,
    max_iter,
    log_interval,
    num_warmup,
    logger=logger
)

results = benchmark.run(repeat_num)
print("\nDetailed Benchmark Results:")
if hasattr(results, 'items'):
    for key, value in results.items():
        print(f"{key}: {value}")
else:
    print(results)

Task: inference
Checkpoint path: /Ziob/343312/AVD-Anonymizer/CrossKD/work_dirs/crosskd_r50_fcos_r101-2x-ms_caffe_fpn_gn-head_2x_ms_coco/epoch_24.pth
Dataset type: test
Max iterations: 100
Log interval: 20
Fuse Conv-BN: True


NameError: name 'nn' is not defined